In [ ]:
# Importieren der benötigten Bibliotheken für Datenverarbeitung, Geodaten, Visualisierung und Karten
import re
import pandas as pd
import geopandas as gpd
from matplotlib import cm, colors
import plotly.express as px
import plotly.graph_objects as go
import folium

# Definieren der Dateipfade für die CSV-Dateien mit den Wetter- und Stationsdaten
file_path_sun_data = 'data\\meteo_swiss_data.csv'
file_path_stn_data = 'data\\meteo_swiss_stn_data.csv'

# Laden der CSV-Dateien in Pandas-DataFrames
sun_data = pd.read_csv(file_path_sun_data, delimiter=';')  # Wetterdaten (Sonnenscheindauer)
stn_data = pd.read_csv(file_path_stn_data, delimiter=';')  # Stationsdaten (Standortinformationen)

# Verknüpfen der Wetterdaten mit den Stationsdaten basierend auf der Stations-ID ('stn')
data = pd.merge(sun_data, stn_data, on='stn', how='inner')

# Konvertieren der Zeitspalte in ein Datumsformat (Jahr und Monat)
data['time'] = pd.to_datetime(data['time'], format='%Y%m')
data['year'] = data['time'].dt.year  # Extrahieren des Jahres
data['month'] = data['time'].dt.month  # Extrahieren des Monats

# Ersetzen von '-' durch '0' und Konvertieren in Float
data['sunhours'] = data['su2000m0'].replace('-', '0').astype(float)

# Entfernen von Einträgen mit ungültigen oder unerwünschten Werten
data = data[(data['sunhours'] > 0) & (data['year'] != 2025) & (data['canton'] != 'LI')]
data['sunhours'] = data['sunhours'].astype(float)  # Sicherstellen, dass die Werte numerisch sind

# Gruppieren der Daten nach Jahr und Monat und Berechnen des monatlichen Durchschnitts der Sonnenscheindauer
monthly_data = data.groupby(['year', 'month'])['sunhours'].mean().unstack(level=0)

# Definieren einer Liste mit deutschen Abkürzungen für die Monatsnamen
month_names = ['Jan', 'Feb', 'Mär', 'Apr', 'Mai', 'Jun', 'Jul', 'Aug', 'Sep', 'Okt', 'Nov', 'Dez']

In [ ]:
# Berechnen des durchschnittlichen Sonnenstunden pro Monat über alle Jahre
average_sunhours = monthly_data.mean(axis=1)

# Berechnen des Medians der Sonnenstunden pro Monat über alle Jahre
median_sunhours = monthly_data.median(axis=1)

# Erstellung eines interaktiven Liniendiagramms mit Plotly
fig = go.Figure()

# Iteriere über die Spalten (Jahre) in den monatlichen Daten und füge eine Linie für jedes Jahr hinzu
for year in monthly_data.columns:
    fig.add_trace(go.Scatter(
        x=monthly_data.index,  # Monatsnamen (Index der Tabelle)
        y=monthly_data[year],  # Sonnenstunden für das jeweilige Jahr
        mode='lines+markers',  # Darstellung als Linie mit Markern
        name=str(year),  # Name des Jahres in der Legende
        opacity=0.5  # Transparenz für bessere Übersichtlichkeit
    ))

# Darstellung des Durchschnitts der Sonnenstunden über alle Jahre
fig.add_trace(go.Scatter(
    x=monthly_data.index,  # Monatsnamen
    y=average_sunhours,  # Durchschnittswerte
    mode='lines',  # Darstellung als Linie
    name='Durchschnitt',  # Name in der Legende
    line=dict(color='black', dash='dash')  # Schwarze gestrichelte Linie
))

# Darstellung des Medians der Sonnenstunden über alle Jahre
fig.add_trace(go.Scatter(
    x=monthly_data.index,  # Monatsnamen
    y=median_sunhours,  # Medianwerte
    mode='lines',  # Darstellung als Linie
    name='Median',  # Name in der Legende
    line=dict(color='red', dash='dot')  # Rote gepunktete Linie
))

# Anpassung des Diagrammlayouts
fig.update_layout(
    title='Durchschnittliche Sonnenstunden pro Monat in der Schweiz',  # Titel des Diagramms
    xaxis_title='Monat',  # Beschriftung der x-Achse
    yaxis_title='Durchschnittliche Sonnenstunden',  # Beschriftung der y-Achse
    xaxis=dict(
        tickmode='array',  # Anpassung der x-Achsen-Ticks
        tickvals=list(range(1, 13)),  # Werte für die Ticks (1 bis 12 für die Monate)
        ticktext=month_names  # Monatsnamen als Beschriftung
    ),
    legend_title='Jahr',  # Titel der Legende
    template='plotly_white'  # Weißes Diagramm-Template
)

# Anzeigen des Diagramms
fig.show()

# Speichern des Diagramms als HTML-Datei
fig.write_html('docs\\assets\\diagramme\\sunhours_per_month.html')

In [ ]:
# Erstellung eines interaktiven Boxplots mit Plotly
# Der Boxplot zeigt die Verteilung der Sonnenstunden pro Monat

# Erstellen des Boxplots
fig_box = px.box(
    data,  # Datensatz mit den Sonnenstunden
    x='month',  # Gruppierung nach Monat
    y='sunhours',  # Werte der Sonnenstunden
    points=False,  # Keine Punkte (Outlier) anzeigen
    labels={'month': 'Monat', 'sunhours': 'Sonnenstunden'}  # Achsenbeschriftungen
)

# Anpassung des Layouts des Boxplots
fig_box.update_layout(
    title='Verteilung der Sonnenstunden pro Monat in der Schweiz',  # Titel des Diagramms
    xaxis=dict(
        tickmode='array',  # Anpassung der x-Achsen-Ticks
        tickvals=list(range(1, 13)),  # Werte für die Ticks (1 bis 12 für die Monate)
        ticktext=month_names  # Monatsnamen als Beschriftung
    ),
    template='plotly_white'  # Weißes Diagramm-Template
)

# Anzeigen des Boxplots
fig_box.show()

# Speichern des Boxplots als HTML-Datei
fig_box.write_html('docs\\assets\\diagramme\\sunhours_distribution_per_month.html')

In [ ]:
# Erstellen einer Karte mit einer Startposition (Schweiz) und Zoomstufe
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Hinzufügen der Schweizer Kantonsgrenzen aus einer GeoJSON-Datei
folium.GeoJson('data\\swiss_cantons.geojson', name='Kantone').add_to(map)

# Funktion zur Umrechnung von Grad, Minuten, Sekunden (GMS) in Dezimalgrad (DG)
def gms_to_dg(gms):
    """
    Konvertiert Koordinaten im Format 'XX°YY' in Dezimalgrad.
    Args:
        gms (str): Koordinaten im Format 'XX°YY'.
    Returns:
        float: Koordinaten in Dezimalgrad.
    Raises:
        ValueError: Wenn das Format ungültig ist.
    """
    match = re.match(r"(\d+)°(\d+)", gms)
    if match:
        grad = int(match.group(1))
        minuten = int(match.group(2))
        return grad + (minuten / 60)
    else:
        raise ValueError("Ungültiges Format. Erwartet: 'XX°YY'")

# Umrechnung der Koordinaten von GMS in Dezimalgrad
stn_data['latitude'] = stn_data['latitude'].apply(gms_to_dg)
stn_data['longitude'] = stn_data['longitude'].apply(gms_to_dg)

# Hinzufügen von Markern für jede Station auf der Karte
for _, stn in stn_data.iterrows():
    folium.Marker(
        location=[stn['latitude'], stn['longitude']],  # Koordinaten der Station
        popup=stn['stn']  # Name der Station als Popup
    ).add_to(map)

# HTML-Code für den Titel
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Verteilung der Wettermessstationen in der Schweiz</b></div>
'''
# Hinzufügen des Titels zur Karte
map.get_root().html.add_child(folium.Element(title_html))

# Speichern der Karte in einer HTML-Datei
map.save('docs\\assets\\diagramme\\swiss_stations_map.html')

# Anzeigen der Karte im Notebook
map

In [ ]:
canton_monthly_avg = data.groupby(['canton_name', 'year', 'month'])['sunhours'].mean().reset_index()

canton_yearly_cum = canton_monthly_avg.groupby(['canton_name', 'year'])['sunhours'].sum().reset_index()
canton_avg = canton_yearly_cum.groupby('canton_name')['sunhours'].mean().reset_index()


# Laden der GeoJSON-Daten der Schweizer Kantone
# Diese Daten enthalten die geografischen Grenzen der Kantone
cantons = gpd.read_file('data\\swiss_cantons.geojson')

# Verknüpfen der Durchschnittswerte mit den GeoJSON-Daten
# Die Verknüpfung erfolgt über den Namen des Kan
# tons
cantons = cantons.merge(canton_avg, left_on='NAME', right_on='canton_name', how='left')

# Erstellung der interaktiven Karte mit Folium
# Festlegen des Startpunkts und der Zoomstufe der Karte
map = folium.Map(location=[46.8, 8.33], zoom_start=7)

# Hinzufügen der Kantonsgrenzen zur Karte
folium.GeoJson('data\\swiss_cantons.geojson', name='Kantone').add_to(map)

# Definition der Farbskala (Blau -> Rot) basierend auf den Durchschnittswerten
norm = colors.Normalize(vmin=canton_avg['sunhours'].min(), vmax=canton_avg['sunhours'].max())
colormap = cm.ScalarMappable(norm=norm, cmap='YlOrRd')

# Hinzufügen von Kreis-Markern für die Durchschnittswerte der Sonnenstunden
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):  # Nur wenn ein Durchschnittswert vorhanden ist
        color = colors.to_hex(colormap.to_rgba(row['sunhours']))  # Farbe basierend auf dem Wert
        folium.Circle(
            location=[row['geometry'].centroid.y, row['geometry'].centroid.x],  # Position des Kreises (Kantonszentrum)
            radius=row['sunhours'] * 7,  # Radius des Kreises (Skalierung basierend auf Sonnenstunden)
            color=color,  # Randfarbe des Kreises
            fill=True,  # Kreis ausfüllen
            fill_color=color,  # Füllfarbe des Kreises
            fill_opacity=0.6,  # Transparenz des Kreises
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"  # Popup mit Kanton und Durchschnittswert
        ).add_to(map)

# Hinzufügen von Text-Markern mit den Durchschnittswerten
for _, row in cantons.iterrows():
    if not pd.isna(row['sunhours']):  # Nur wenn ein Durchschnittswert vorhanden ist
        folium.map.Marker(
            [row['geometry'].centroid.y, row['geometry'].centroid.x],  # Position des Textes (Kantonszentrum)
            icon=folium.DivIcon(
                icon_size=(15, 12),  # Größe des Icons
                icon_anchor=(20, 12),  # Ankerpunkt des Icons
                html=f'<div style="font-size: 12pt; color: black;">{int(row["sunhours"])}</div>'  # Text mit Sonnenstunden
            ),
            popup=f"{row['NAME']}: {row['sunhours']:.2f}"  # Popup mit Kanton und Durchschnittswert
        ).add_to(map)

# HTML-Code für den Titel
title_html = '''
<div style="font-size:20px;position: absolute;z-index: 1000;left: 25%;"><b>Durchschnittliche jährliche Sonnenstunden pro Kanton</b></div>
'''
# Hinzufügen des Titels zur Karte
map.get_root().html.add_child(folium.Element(title_html))


# Speichern der Karte in einer HTML-Datei
# Die Karte wird in einem Ordner gespeichert, um sie später im Browser anzuzeigen
map.save('docs\\assets\\diagramme\\swiss_sunhours_map.html')

# Anzeigen der Karte (optional, wenn in einer Jupyter-Umgebung verwendet)
map